## DIA 052: Implementación de un Sistema de Autenticación con OAuth 2.0 en Flask

Hoy implementaremos OAuth 2.0 en nuestra API Flask, permitiendo a los usuarios autenticarse mediante Google en lugar de usar credenciales tradicionales. Esto mejora la seguridad y experiencia del usuario, facilitando el inicio de sesión sin necesidad de recordar contraseñas.

OAuth 2.0 es el estándar moderno para autenticación y autorización, usado por servicios como Google, Facebook y GitHub.

✅ Beneficios de OAuth 2.0:

Mayor seguridad al no almacenar contraseñas.
Inicio de sesión rápido con Google OAuth.
Acceso a información del usuario con su consentimiento.
🖥️ Código Completo (api.py)
python
Copiar
Editar
import os
import json
import logging
from flask import Flask, redirect, url_for, session, jsonify
from authlib.integrations.flask_client import OAuth
from flask_sqlalchemy import SQLAlchemy
from flask_jwt_extended import JWTManager, create_access_token

# Configuración básica
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'supersecretkey')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'jwtsecretkey')

# Inicialización de extensiones
db = SQLAlchemy(app)
jwt = JWTManager(app)
oauth = OAuth(app)

# Configuración de Google OAuth
GOOGLE_CLIENT_ID = os.getenv("GOOGLE_CLIENT_ID")
GOOGLE_CLIENT_SECRET = os.getenv("GOOGLE_CLIENT_SECRET")

oauth.register(
    name="google",
    client_id=GOOGLE_CLIENT_ID,
    client_secret=GOOGLE_CLIENT_SECRET,
    access_token_url="https://oauth2.googleapis.com/token",
    authorize_url="https://accounts.google.com/o/oauth2/auth",
    api_base_url="https://www.googleapis.com/oauth2/v1/",
    client_kwargs={"scope": "openid email profile"},
)

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# ---------------------------
# Modelo de Usuario
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(120), unique=True, nullable=False)
    name = db.Column(db.String(100), nullable=False)

# ---------------------------
# Endpoints de Autenticación
# ---------------------------
@app.route('/login')
def login():
    """
    Redirige a la página de autenticación de Google.
    """
    return oauth.google.authorize_redirect(url_for("authorize", _external=True))

@app.route('/authorize')
def authorize():
    """
    Maneja la respuesta de Google OAuth y autentica al usuario.
    """
    token = oauth.google.authorize_access_token()
    user_info = oauth.google.get("userinfo").json()

    if not user_info:
        return jsonify({"error": "No se pudo obtener la información del usuario"}), 400

    email = user_info["email"]
    name = user_info["name"]

    user = User.query.filter_by(email=email).first()
    if not user:
        user = User(email=email, name=name)
        db.session.add(user)
        db.session.commit()

    # Generar token JWT
    access_token = create_access_token(identity=email)
    
    return jsonify({"msg": "Inicio de sesión exitoso", "access_token": access_token, "user": user_info})

@app.route('/profile')
def profile():
    """
    Devuelve el perfil del usuario autenticado.
    """
    if "user" not in session:
        return jsonify({"error": "No autenticado"}), 401
    return jsonify(session["user"])

# ---------------------------
# Ejecutar la aplicación
# ---------------------------
if __name__ == '__main__':
    app.run(debug=True)
🔍 Explicación de las Principales Implementaciones
🔹 🔑 Configuración de OAuth con Google

Se registra un cliente OAuth con authlib.
Se configuran los endpoints de autenticación de Google.
🔹 🔄 Flujo de Autenticación OAuth 2.0
✅ /login → Redirige al usuario a Google para autenticación.
✅ /authorize → Google devuelve un token, obtenemos el perfil y generamos un JWT.
✅ /profile → Devuelve la información del usuario autenticado.

🔹 🔐 Seguridad Mejorada

El usuario no necesita ingresar contraseñas en nuestra API.
Se usa un token JWT para autenticar solicitudes futuras.
🔹 📂 Persistencia en Base de Datos

Se almacena el email y nombre del usuario en la base de datos.
Si el usuario ya existe, se reutiliza su cuenta.
